# Jaana's quest

Noniin, testi tässä. Tulokset **eivät** kovin mairittelevia ole. Syytän osaksi Jupyteria, joka jostain syystä päätti lopettaa toimintansa tunnin jälkeen, jonka jälkeen menin muutaman tunnin sokkona kunnes sain softan heräämään.

Malli on treenattu huikealla 8% tarkkuudella viimeisen tunnin sisällä, jona aikana lähes kaikki dokumentissa oleva sisältö on tehty.

Ratkaisu on treenautettu vuoden 2018 asematiedoilla, eli parsittu datasta ainoastaan HKI-TRE reitillä olevien asemien `differenceInMinutes` -tietue, joka siis indikoi junan sen hetkisen aikataulun. Parempiakin ratkaisuja varmasti olisi, mutta (kiireessä) päädyin tähän. Optimointia ei ole suoritettu ollenkaan (Ja logistinen regressiomalli ei välttämättä myöskään se paras tähän ole...), esim. paremmalla / siivotummalla datasetillä, _gradient descent_ metodilla etc. tulokset olisivat varmasti parempia.

Datan puolesta lisäisin siihen ainakin sääolosuhteet, koska ne varmasti ovat pääsyy aikataulujen muutoksiin.

Kiitokset.

-Ville R.



In [370]:
from datetime import date, datetime, timedelta
import pandas as pd
import numpy as np
import requests
import pickle

# Where to stop to in the TimeTableRows
TPE_SLICE = 90
API_URL = 'https://rata.digitraffic.fi/api/v1/trains'
START_DATE = date(2018, 1, 1)
END_DATE = date(2018, 8, 22)

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

def reshape(df):
    cols = df.stationShortCode.tolist()
    row = df.differenceInMinutes.tolist()
    new_df = pd.DataFrame([row], columns=cols)
    
    return new_df

def get_data(date, train = 45):
    """
    Get data from API
    """
    import json
    
    # If the file already exists => do not add headers
    add_headers = False
    
    try:
        file = open('data.csv', 'r')
    except IOError:
        file = open('data.csv', 'w')
        add_headers = True
    file.close()
    
    try:
        url = '{0}/{1}/{2}'.format(API_URL, date, train)
        print('getting data for date {}'.format(date))
        r = requests.get(url, headers={'Cache-Control': 'no-cache'})
        data = r.json()
        ttRows = json.dumps(data[0].get('timeTableRows')[0:TPE_SLICE])
        df = pd.read_json(ttRows)

        shaped = reshape(df)

        return shaped.to_csv('data.csv', mode='a', header=add_headers)

    except Exception as e:
        print(e)

# Comment for getting new data
#for d in daterange(START_DATE, END_DATE):
#    get_data(d.strftime("%Y-%m-%d"))

In [368]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

df = pd.read_csv('data.csv')

#df.shape
#df.describe()

X_train, X_test, y_train, y_test = train_test_split(df.drop('TPE',axis=1),df.TPE, random_state=42)

# Lez training-montage =>
model = LogisticRegression()
model.fit(X_train,y_train)

pred = model.predict(X_test)
model.score(X_test, y_test)

0.0847457627118644

In [371]:
with open('mdl.pkl', 'wb') as fid:
    pickle.dump(model, fid,2) 